# Example Usage of Thursday API

In [1]:
import h5py

import keras 
from keras.utils import to_categorical
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from format_data import get_data, get_fr_data, augment_data, resize_array
from models import SklearnModel, HOGNet

C:\Users\fires\Anaconda3\envs\CSIRO\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Path to data output by fri_frii_download.ipynb.
fr_data_path = "data/data.h5"
# Path to Aniyan+17 validation set.
aniyan_path = "data/asu.tsv"

seed = 0

# Getting indices of training and testing data 

We use the get_data function to get locations of the training and testing data from our FR data and Aniyan data

Coordinates from data.h5 and asu.tsv are converted into Skycoords. The two are cross-matched. Sources within five arcsec radius of each other are assumed to be the same object. Aniyan sources are extracted from the FR sources, and formated into class-wise balanced training set. The remainder are formated into class-wise balanced testing set.

In [3]:
train_i, test_i = get_data(fr_data_path, aniyan_path, seed=seed)

# Constructing the data generator 
Using the augment_data construxt our keras.preprocessing.image.ImageDataGenerator object to apply randomly augmentations to our data

In [4]:
datagen = augment_data(rotation_range=180, zoom_range=0.2, shift_range=0.0, flip=True)

# Instantiating Sklearn Classifier
We use the SklearnModel class to construct our random forest classifer. Any other Sklearn Classifier can be used.

In [5]:
rft = SklearnModel(LogisticRegression, datagen=datagen, nb_augment=10, seed=seed)

# Training Classifer
We train our classifer with the fit method

In [7]:
with h5py.File(fr_data_path, 'r') as data:
    train_x = np.asarray(data['images'])[train_i]
    train_y = np.asarray(data['labels'])[train_i]
    
    # Formatting images
    train_x = resize_array(train_x, dim=256)
    train_x = np.expand_dims(train_x, axis=3)
    
    rft.fit(train_x, train_y)

# Saving Classifer to Pickle File
Using the save method

# Training Classifer
We train our classifer with the fit method

In [6]:
rft.save(path="data/rft.pk")

# Classifying Unseen Samples
First, we load our model with the load method (If the save method has been used on the same script, path is inferred). We test our classifier with the predict method.

In [7]:
rft.save(path="data/rft.pk")

# Classifying Unseen Samples
First, we load our model with the load method (If the save method has been used on the same script, path is inferred). We test our classifier with the predict method.

In [8]:
rtf = rft.load()

with h5py.File(fr_data_path, 'r') as data:
    test_x = np.asarray(data['images'])[test_i]
    test_y = np.asarray(data['labels'])[test_i]
    
    # Formatting images
    test_x = resize_array(test_x, dim=256)
    test_x = np.expand_dims(test_x, axis=3)
    
    predictions = rtf.predict(test_x)
    
    correct = test_y == predictions
    print('Accuracy: {:.02%}'.format(correct.mean()))

Accuracy: 100.00%


# Instantiating HOGNet Model
We use the HOGNet class to construct our custom keras model.

In [ ]:
hognet = HOGNet(datagen=datagen, batch_size=10, steps_per_epoch=50, max_epoch=1, patience=5, seed=seed)

# Training Classifer
We train our classifer with the fit method. Model stops training when loss stop decreasing for a set number of epochs (patience argument controls this) or when it reaches the maximum number of epochs. The amount of data augmented by datagen is dictated by the batch_size, and how many batches are generated per epoch is controlled by steps_per_epoch.

In [9]:
hognet = HOGNet(datagen=datagen, batch_size=10, steps_per_epoch=50, max_epoch=1, patience=5, seed=seed)

C:\Users\fires\Desktop\CSIRO\thursday\thursday\models.py:422: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(8, kernel_size=(1, 1), strides=(1, 1), activation="relu", trainable=True, name="votes", use_bias=False)`
  trainable=True, bias=False, name="votes")(sin_cos_vec)


hognet.fit(train_x, train_y)

# Saving Model Weights to H5py file
Using the save method

In [10]:
hognet.save(path="data/hognet.h5")

Epoch 1/1
50/50 [==============================] - 53s 1s/step - loss: 15.9424 - acc: 0.0000e+00


# Classifying Unseen Samples
First, we load our weights into the model with the load method (If the save method has been used on the same script, path is inferred). We test our classifier with the predict method.

In [11]:
hognet.save(path="data/hognet.h5")

hognet.load()

predictions = hognet.predict(test_x)

correct = test_y == predictions
print('Accuracy: {:.02%}'.format(correct.mean()))